In [1]:
import pandas as pd
from pennylane import numpy as np
from Factory.data import PrepareData
from Factory.model import QBC
from sklearn import metrics

C:\Users\jonas\Anaconda3\envs\my-rdkit-env\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


# Prepare Data

Load data in

In [2]:
df = pd.read_csv('fraud_detection_bank_dataset.csv', sep=',')
df = df.astype(float)
df = df.drop(['Unnamed: 0'], axis = 1)

In [3]:
df

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_103,col_104,col_105,col_106,col_107,col_108,col_109,col_110,col_111,targets
0,9.0,1354.0,0.0,18.0,0.0,1.0,7.0,9.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,49.0,1.0
1,0.0,239.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,55.0,1.0
2,0.0,260.0,0.0,4.0,0.0,3.0,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,56.0,1.0
3,17.0,682.0,0.0,1.0,0.0,0.0,8.0,17.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,65.0,1.0
4,1.0,540.0,0.0,2.0,0.0,1.0,7.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,175.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20463,0.0,88.0,0.0,0.0,0.0,2.0,-1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,85.0,0.0
20464,0.0,134.0,0.0,2.0,0.0,0.0,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0
20465,4.0,393.0,1.0,1.0,0.0,0.0,-1.0,4.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,0.0
20466,0.0,10.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0


Clean data

In [4]:
df_clean = df.loc[:, df.any()]
df_clean.describe()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_13,col_14,...,col_103,col_104,col_105,col_106,col_107,col_108,col_109,col_110,col_111,targets
count,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000,...,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000,20468.000000
mean,3.226256,294.793043,0.420021,2.329343,0.083594,0.939857,2.386066,3.226256,0.038646,0.363152,...,0.004935,0.358120,0.002443,0.308384,0.190737,0.000049,0.049345,0.024282,43.486125,0.265683
std,20.564308,717.541984,7.367275,10.068512,0.840537,4.222896,3.018140,20.564308,0.192754,0.480920,...,0.073478,0.479459,0.049366,0.461837,0.392892,0.006990,0.216593,0.305079,59.217560,0.441707
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,38.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000
50%,0.000000,97.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19.000000,0.000000
75%,2.000000,283.000000,0.000000,2.000000,0.000000,1.000000,6.000000,2.000000,0.000000,1.000000,...,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,61.250000,1.000000
max,2301.000000,37808.000000,904.000000,772.000000,54.000000,230.000000,11.000000,2301.000000,1.000000,1.000000,...,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,17.000000,747.000000,1.000000


In [5]:
df_clean = df_clean.drop('col_7', axis = 1)

In [6]:
df_clean = df_clean.dropna()

In [7]:
df_clean["col_28"].describe()

count    20468.000000
mean         0.999951
std          0.006990
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: col_28, dtype: float64

Prepare data

In [8]:
data = PrepareData(data = df_clean,target = "targets", sample_size = 4000, test_split = 0.3, seed = 10)

In [9]:
data.view_info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2800 entries, 738 to 14645
Data columns (total 91 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   col_0    2800 non-null   float64
 1   col_1    2800 non-null   float64
 2   col_2    2800 non-null   float64
 3   col_3    2800 non-null   float64
 4   col_4    2800 non-null   float64
 5   col_5    2800 non-null   float64
 6   col_6    2800 non-null   float64
 7   col_13   2800 non-null   float64
 8   col_14   2800 non-null   float64
 9   col_15   2800 non-null   float64
 10  col_16   2800 non-null   float64
 11  col_17   2800 non-null   float64
 12  col_22   2800 non-null   float64
 13  col_23   2800 non-null   float64
 14  col_24   2800 non-null   float64
 15  col_25   2800 non-null   float64
 16  col_26   2800 non-null   float64
 17  col_27   2800 non-null   float64
 18  col_29   2800 non-null   float64
 19  col_30   2800 non-null   float64
 20  col_31   2800 non-null   float64
 21  col_32   28

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_13,col_14,col_15,...,col_103,col_104,col_105,col_106,col_107,col_108,col_109,col_110,col_111,targets
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,...,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,3.016000,276.914500,0.381750,2.232000,0.073750,0.979750,2.397500,0.037000,0.364000,0.323000,...,0.004250,0.355000,0.001000,0.303750,0.189250,0.000250,0.051500,0.030500,43.570250,0.269750
std,12.684497,606.945801,3.353107,7.274435,0.485664,4.250807,3.022297,0.188785,0.481209,0.467681,...,0.065061,0.478573,0.031611,0.459933,0.391756,0.015811,0.221043,0.390651,60.558277,0.443885
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000
50%,0.000000,95.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19.000000,0.000000
75%,2.000000,272.250000,0.000000,2.000000,0.000000,1.000000,6.000000,0.000000,1.000000,1.000000,...,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,62.000000,1.000000
max,342.000000,12980.000000,162.000000,206.000000,20.000000,143.000000,11.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,13.000000,747.000000,1.000000


# Use LDA

Preprocess LDA

In [10]:
data.x_train["col_69"]

738      0.0
10760    0.0
12934    9.0
6958     6.0
1255     1.0
        ... 
16509    2.0
19140    1.0
18861    0.0
6091     1.0
14645    2.0
Name: col_69, Length: 2800, dtype: float64

In [11]:
data.perform_LDA()

Performing LDA...


C:\Users\jonas\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jonas\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
data.view_info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2800 entries, 738 to 14645
Data columns (total 91 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   col_0    2800 non-null   float64
 1   col_1    2800 non-null   float64
 2   col_2    2800 non-null   float64
 3   col_3    2800 non-null   float64
 4   col_4    2800 non-null   float64
 5   col_5    2800 non-null   float64
 6   col_6    2800 non-null   float64
 7   col_13   2800 non-null   float64
 8   col_14   2800 non-null   float64
 9   col_15   2800 non-null   float64
 10  col_16   2800 non-null   float64
 11  col_17   2800 non-null   float64
 12  col_22   2800 non-null   float64
 13  col_23   2800 non-null   float64
 14  col_24   2800 non-null   float64
 15  col_25   2800 non-null   float64
 16  col_26   2800 non-null   float64
 17  col_27   2800 non-null   float64
 18  col_29   2800 non-null   float64
 19  col_30   2800 non-null   float64
 20  col_31   2800 non-null   float64
 21  col_32   28

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_13,col_14,col_15,...,col_103,col_104,col_105,col_106,col_107,col_108,col_109,col_110,col_111,targets
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,...,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,3.016000,276.914500,0.381750,2.232000,0.073750,0.979750,2.397500,0.037000,0.364000,0.323000,...,0.004250,0.355000,0.001000,0.303750,0.189250,0.000250,0.051500,0.030500,43.570250,0.269750
std,12.684497,606.945801,3.353107,7.274435,0.485664,4.250807,3.022297,0.188785,0.481209,0.467681,...,0.065061,0.478573,0.031611,0.459933,0.391756,0.015811,0.221043,0.390651,60.558277,0.443885
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000
50%,0.000000,95.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19.000000,0.000000
75%,2.000000,272.250000,0.000000,2.000000,0.000000,1.000000,6.000000,0.000000,1.000000,1.000000,...,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,62.000000,1.000000
max,342.000000,12980.000000,162.000000,206.000000,20.000000,143.000000,11.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,13.000000,747.000000,1.000000


Define and train model

In [13]:
model_LDA = QBC(data, n_dim = 2, n_layers = 5, backend = "lightning.qubit")

In [14]:
model_LDA.train()

New best
Iter:     1 | Cost: 0.7942621 | Accuracy: 0.7342857 
New best
Iter:     2 | Cost: 0.6010749 | Accuracy: 0.8164286 
New best
Iter:     3 | Cost: 0.5796202 | Accuracy: 0.8192857 
Iter:     4 | Cost: 0.5408751 | Accuracy: 0.8160714 
New best
Iter:     5 | Cost: 0.5182431 | Accuracy: 0.8289286 
New best
Iter:     6 | Cost: 0.4991535 | Accuracy: 0.8467857 
New best
Iter:     7 | Cost: 0.5013836 | Accuracy: 0.8521429 
New best
Iter:     8 | Cost: 0.5088154 | Accuracy: 0.8532143 
New best
Iter:     9 | Cost: 0.4736870 | Accuracy: 0.8539286 
Iter:    10 | Cost: 0.4708895 | Accuracy: 0.8500000 
Iter:    11 | Cost: 0.4977035 | Accuracy: 0.8275000 
Iter:    12 | Cost: 0.5057134 | Accuracy: 0.8217857 
Iter:    13 | Cost: 0.4852301 | Accuracy: 0.8296429 
Iter:    14 | Cost: 0.4718846 | Accuracy: 0.8396429 
Iter:    15 | Cost: 0.4545640 | Accuracy: 0.8532143 
Iter:    16 | Cost: 0.4554596 | Accuracy: 0.8510714 
Iter:    17 | Cost: 0.4633778 | Accuracy: 0.8485714 
Iter:    18 | Cost: 0.46220

Get the test set from the data to predict:

In [15]:
_, _, X_test, Y_test = data.get_preprocessed()

In [16]:
predictions_LDA = model_LDA.predict(X_test)

In [17]:
print(metrics.classification_report(predictions_LDA,Y_test))
print(metrics.precision_score(predictions_LDA,Y_test))
print(metrics.recall_score(predictions_LDA,Y_test))
print(metrics.f1_score(predictions_LDA,Y_test))
print(metrics.balanced_accuracy_score(predictions_LDA,Y_test))
print(metrics.confusion_matrix(predictions_LDA,Y_test))

              precision    recall  f1-score   support

        -1.0       0.97      0.86      0.91       991
         1.0       0.57      0.86      0.68       209

    accuracy                           0.86      1200
   macro avg       0.77      0.86      0.80      1200
weighted avg       0.90      0.86      0.87      1200

0.5682539682539682
0.8564593301435407
0.683206106870229
0.8596121070495706
[[855 136]
 [ 30 179]]


# Use PCA

Preprocess PCA

In [18]:
data.x_train

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_13,col_14,col_15,...,col_101,col_102,col_103,col_104,col_105,col_106,col_107,col_109,col_110,col_111
738,0.0,18.0,0.0,0.0,0.0,1.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10760,11.0,740.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12934,0.0,382.0,0.0,24.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,194.0
6958,2.0,136.0,1.0,0.0,4.0,0.0,-1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,64.0
1255,0.0,26.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16509,0.0,322.0,0.0,6.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
19140,3.0,78.0,0.0,2.0,0.0,0.0,7.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
18861,4.0,78.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6091,1.0,175.0,0.0,2.0,0.0,0.0,7.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0


In [19]:
data.x_test

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_13,col_14,col_15,...,col_101,col_102,col_103,col_104,col_105,col_106,col_107,col_109,col_110,col_111
4181,0.0,34.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7592,0.0,5.0,0.0,1.0,0.0,0.0,-1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33,45.0,618.0,0.0,1.0,0.0,10.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,181.0
18507,7.0,340.0,0.0,2.0,0.0,0.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,22.0
9106,2.0,36.0,0.0,1.0,0.0,0.0,6.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9822,2.0,375.0,0.0,2.0,0.0,0.0,6.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,20.0
19204,2.0,190.0,0.0,1.0,0.0,3.0,2.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,25.0
12449,0.0,23.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16268,0.0,209.0,0.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,154.0


In [20]:
data.perform_PCA()

Performing PCA...


In [21]:
data.view_info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2800 entries, 738 to 14645
Data columns (total 91 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   col_0    2800 non-null   float64
 1   col_1    2800 non-null   float64
 2   col_2    2800 non-null   float64
 3   col_3    2800 non-null   float64
 4   col_4    2800 non-null   float64
 5   col_5    2800 non-null   float64
 6   col_6    2800 non-null   float64
 7   col_13   2800 non-null   float64
 8   col_14   2800 non-null   float64
 9   col_15   2800 non-null   float64
 10  col_16   2800 non-null   float64
 11  col_17   2800 non-null   float64
 12  col_22   2800 non-null   float64
 13  col_23   2800 non-null   float64
 14  col_24   2800 non-null   float64
 15  col_25   2800 non-null   float64
 16  col_26   2800 non-null   float64
 17  col_27   2800 non-null   float64
 18  col_29   2800 non-null   float64
 19  col_30   2800 non-null   float64
 20  col_31   2800 non-null   float64
 21  col_32   28

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_13,col_14,col_15,...,col_103,col_104,col_105,col_106,col_107,col_108,col_109,col_110,col_111,targets
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,...,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,3.016000,276.914500,0.381750,2.232000,0.073750,0.979750,2.397500,0.037000,0.364000,0.323000,...,0.004250,0.355000,0.001000,0.303750,0.189250,0.000250,0.051500,0.030500,43.570250,0.269750
std,12.684497,606.945801,3.353107,7.274435,0.485664,4.250807,3.022297,0.188785,0.481209,0.467681,...,0.065061,0.478573,0.031611,0.459933,0.391756,0.015811,0.221043,0.390651,60.558277,0.443885
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000
50%,0.000000,95.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19.000000,0.000000
75%,2.000000,272.250000,0.000000,2.000000,0.000000,1.000000,6.000000,0.000000,1.000000,1.000000,...,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,62.000000,1.000000
max,342.000000,12980.000000,162.000000,206.000000,20.000000,143.000000,11.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,13.000000,747.000000,1.000000


Define and train model

In [22]:
model_PCA = QBC(data, n_dim = 2, n_layers = 5, backend = "lightning.qubit")

In [23]:
model_PCA.train()

New best
Iter:     1 | Cost: 0.9093360 | Accuracy: 0.7242857 
New best
Iter:     2 | Cost: 0.8465034 | Accuracy: 0.7257143 
New best
Iter:     3 | Cost: 0.8082671 | Accuracy: 0.7271429 
Iter:     4 | Cost: 0.8092411 | Accuracy: 0.7271429 
Iter:     5 | Cost: 0.7952753 | Accuracy: 0.7271429 
Iter:     6 | Cost: 0.7996801 | Accuracy: 0.7271429 
Iter:     7 | Cost: 0.8025520 | Accuracy: 0.7271429 
Iter:     8 | Cost: 0.8014394 | Accuracy: 0.7271429 
Iter:     9 | Cost: 0.7855185 | Accuracy: 0.7271429 
Iter:    10 | Cost: 0.7721740 | Accuracy: 0.7271429 
Iter:    11 | Cost: 0.8083123 | Accuracy: 0.7253571 
Iter:    12 | Cost: 0.8672649 | Accuracy: 0.7221429 
Iter:    13 | Cost: 0.8521487 | Accuracy: 0.7239286 
Iter:    14 | Cost: 0.8174532 | Accuracy: 0.7253571 
Iter:    15 | Cost: 0.7822240 | Accuracy: 0.7271429 
Iter:    16 | Cost: 0.7806030 | Accuracy: 0.7271429 
Iter:    17 | Cost: 0.7815206 | Accuracy: 0.7271429 
Iter:    18 | Cost: 0.7857112 | Accuracy: 0.7271429 
Iter:    19 | Cost:

Get the test set from the data to predict:

In [24]:
predictions_PCA = model_PCA.predict(X_test)

In [25]:
print(metrics.classification_report(predictions_PCA,Y_test))
print(metrics.precision_score(predictions_PCA,Y_test))
print(metrics.recall_score(predictions_PCA,Y_test))
print(metrics.f1_score(predictions_PCA,Y_test))
print(metrics.balanced_accuracy_score(predictions_PCA,Y_test))
print(metrics.confusion_matrix(predictions_PCA,Y_test))

              precision    recall  f1-score   support

        -1.0       0.86      0.83      0.84       920
         1.0       0.49      0.55      0.52       280

    accuracy                           0.76      1200
   macro avg       0.68      0.69      0.68      1200
weighted avg       0.77      0.76      0.77      1200

0.49206349206349204
0.5535714285714286
0.5210084033613446
0.6898291925465838
[[760 160]
 [125 155]]
